# Assignment 07: Mustaeen Ahmed

In [1]:
import pygame
import moderngl
from pyglm import glm
from loadModelUsingAssimp_V1 import getObjectDataList

pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()

WIDTH = 840
HEIGHT = 480

clock = pygame.time.Clock()
running = True

pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

In [3]:
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.set_mode((WIDTH, HEIGHT), flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 07: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info['GL_VERSION']

'3.3.0 NVIDIA 555.99'

In [4]:
geom_data_list, index_data_list, bound, tex_names, scene = getObjectDataList("mario_obj/scene.gltf", True)

SCENE:
number of meshes: 8
number of materials: 9
Mesh Name:  Object_0
material Index:  0
number of vertices:  1801
number of faces:  3322
number of indices:  9966
first 3 verts:
[ 0.29 -1.18  6.59]
minimum:  -1.815517
first 3 indices:
[0 1 2]
number of Texture coordinate arrays:  1
number of Texture coordinates:  3602
first 3 texture coordinates:
[2.94 1.23 2.95]
Mesh Name:  Object_1
material Index:  1
number of vertices:  3496
number of faces:  6616
number of indices:  19848
first 3 verts:
[-0.67 -0.91  5.68]
minimum:  -1.465921
first 3 indices:
[0 1 2]
number of Texture coordinate arrays:  1
number of Texture coordinates:  6992
first 3 texture coordinates:
[1.75 1.74 1.76]
Mesh Name:  Object_2
material Index:  2
number of vertices:  3738
number of faces:  6380
number of indices:  19140
first 3 verts:
[2.65 0.09 4.35]
minimum:  -3.850452
first 3 indices:
[0 1 2]
number of Texture coordinate arrays:  1
number of Texture coordinates:  7476
first 3 texture coordinates:
[5.41 1.98 5.41]


In [5]:
vertex_shader_code = '''
    #version 330 core
    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;

    uniform mat4 model, view, perspective;
    out vec2 f_uv;

    void main() {
        f_uv = uv;
        vec4 P = perspective * view * model * vec4(position, 1);
        gl_Position = P;
    }
'''

fragment_shader_code = '''
    #version 330 core
    in vec2 f_uv;
    uniform sampler2D map;
    out vec4 color;

    void main() {
        color = texture(map, f_uv);
    }
'''

In [6]:
program = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

renderables = []
for i, geom_data in enumerate(geom_data_list):
    geom_buffer = gl.buffer(geom_data)
    index_buffer = gl.buffer(index_data_list[i])

    renderable = gl.vertex_array(program,
        [(geom_buffer, "3f 2f", "position", "uv")], index_buffer=index_buffer, index_element_size=4                             
    )

    renderables.append(renderable)

In [7]:
tex_names

['mario_obj/textures/submesh_0_baseColor.png',
 'mario_obj/textures/submesh_1_baseColor.png',
 'mario_obj/textures/submesh_2_baseColor.png',
 'mario_obj/textures/submesh_3_baseColor.png',
 'mario_obj/textures/submesh_4_baseColor.png',
 'mario_obj/textures/submesh_5_baseColor.png',
 'mario_obj/textures/submesh_6_baseColor.png',
 'mario_obj/textures/submesh_7_baseColor.png']

In [8]:
for i, tex_name in enumerate(tex_names):
    texture_img = pygame.image.load(tex_name)
    texture_data = pygame.image.tobytes(texture_img, "RGB", True)
    texture = gl.texture(texture_img.get_size(), data=texture_data, components=3)
    texture.build_mipmaps()
    sampler = gl.sampler(texture=texture, filter=(gl.LINEAR_MIPMAP_LINEAR, gl.LINEAR), repeat_x=True, repeat_y=True)
    sampler.use(i)

In [9]:
displacement_vector = 2 * bound.radius * glm.vec3(0, 0, 1)

target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0, 1, 0)

fov_radian = glm.radians(45)
aspect = WIDTH / HEIGHT
near = bound.radius
far = 3 * bound.radius
perspective_matrix = glm.perspective(fov_radian, aspect, near, far)

In [ ]:
def recursive_render(node, M):
    if node is None:
        print("Node is empty")
        return
    
    node_transform = glm.transpose(glm.mat4(node.transformation))
    current_transform = M * node_transform

    if node.num_meshes > 0:
        for index in node.mesh_indices:
            program['map'] = index
            program['model'].write(current_transform)
            renderables[index].render()

    for node in node.children:
        recursive_render(node, current_transform)

def render():
    recursive_render(scene.root_node, glm.mat4(1))

running = True
clock = pygame.time.Clock()
alpha = 0
pause = True
gl.enable(gl.DEPTH_TEST)

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif (event.type == pygame.WINDOWRESIZED):
                width = event.x
                height = event.y
                perspective_matrix = glm.perspective(fov_radian, width / height, near, far)


    eye_point = glm.vec3(bound.center) + glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0, 1, 0))
    view_matrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.5, 0.5, 0.0)

    program["view"].write(view_matrix)
    program["perspective"].write(perspective_matrix)

    render()

    pygame.display.flip()

    if not pause:
        clock.tick(60)
        alpha = alpha + 1
        if alpha > 360:
            alpha = 0

pygame.display.quit()